### An example implementation of the CAREforMe pipeline using the APD dataset

In [36]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join("../care_for_me"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import numpy as np
import pandas as pd
import apd

from care_for_me import signals
from sklearn.metrics import accuracy_score, f1_score


ROOT_DIR = "/Users/emilyzhou/Desktop/Research/CAREForMe/"
DATA_DIR = os.path.join(ROOT_DIR, "data")
APD_PATH = os.path.join(DATA_DIR, "APD")
SOURCE_FOLDER = os.path.join(APD_PATH, "formatted")
METRICS = os.path.join(DATA_DIR, "metrics", "APD")

ALL = "all"
HA = "high_anxiety_group"
LA = "low_anxiety_group"

ha_participant_indices = [
    '4', '6', '7', '8', '10', '12', '15', '16', '18', '22', '26', '27', '29', '31', '32', '33', '35', '42', '45', '47', '48', '49', '54', '55', '66', '69'
]

la_participant_indices = [
    '14', '21', '23', '25', '34', '39', '43', '46', '51', '57', '71', '72', '77', '78', '79', '80', '82', '83', '84', '85', '87', '88', '89', '91', '92', '93'
]

SUBJECTS = ha_participant_indices.extend(la_participant_indices)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Format data to be compatible with CAREforMe pipeline
# Only needs to be run once locally
# Status: COMPLETE, do not re-run
apd.reformat_and_save_data()

In [38]:
# labels = apd.get_suds_labels()

# TODO: Generate binary stress labels from APD self-reports instead of 
# random labels
def generate_labels(data):
    size = data.shape[0]
    labels = np.random.randint(2, size=size)

    return labels, data

### Binary stress classification

In [40]:
# Build pipeline with default preprocessing and feature extraction methods

from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.label_generator.label_generator import LabelGenerator
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Pipeline

from sklearn.svm import SVC


signal_types = [
    signals.Signals.ECG,
    # signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

label_gen = generate_labels
signal_acq = SignalAcquisition(source_folder=SOURCE_FOLDER, signal_types=signal_types)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=100)
feature_extractor = FeatureExtractor()
label_generator = LabelGenerator(label_generation_method=label_gen)

model = SVC()
feature_selector = FeatureSelector(model, feature_names, num_features=3)
estimator_train = Estimator(0, model, name="SVC training")
estimator_test = Estimator(1, model, name="SVC testing")
estimator_train_val_test = Estimator(2, model, name="SVC train-val-test")

pipeline = Pipeline()


pipeline.generate_nodes_from_layers(
    # [signal_acq, signal_preprocessor, feature_extractor]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train, estimator_test]
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train_val_test]
)

out = pipeline.run()

# Results
# fitted_model = out[0]
y_true = out[1]
y_pred = out[2]

acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

print(f"\nAccuracy: {acc}")
# print(f"F1-score: {f1}")

Running node Signal Acquisition...
- Elapsed time: 0.0 s
Running node Signal Preprocessor...
- Elapsed time: 10.6 s
Running node Feature Extractor...


100%|██████████| 52/52 [00:00<00:00, 3201.19it/s]

- Elapsed time: 0.019 s
Running node Label Generator...
- Elapsed time: 0.0 s
Running node Feature Selector...


- Elapsed time: 0.549 s
Running node SVC train-val-test...
Cross-validation scores: [0.54794521 0.56164384 0.57534247 0.54794521 0.56944444]
- Elapsed time: 0.022 s

Accuracy: 0.43478260869565216


### Subject identification

In [ ]:
# Build pipeline with default preprocessing and feature extraction methods

from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.label_generator.label_generator import LabelGenerator
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Pipeline

from sklearn.svm import SVC


signal_types = [
    signals.Signals.ECG,
    # signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

label_gen = "subject"
signal_acq = SignalAcquisition(source_folder=SOURCE_FOLDER, signal_types=signal_types)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=100)
feature_extractor = FeatureExtractor()
label_generator = LabelGenerator(label_generation_method=label_gen)

model = SVC()
feature_selector = FeatureSelector(model, feature_names, num_features=3)
estimator_train = Estimator(0, model, name="SVC training")
estimator_test = Estimator(1, model, name="SVC testing")
estimator_train_val_test = Estimator(2, model, name="SVC train-val-test")

pipeline = Pipeline()


pipeline.generate_nodes_from_layers(
    # [signal_acq, signal_preprocessor, feature_extractor]
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train, estimator_test]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train_val_test]
)

out = pipeline.run()

# Results
# fitted_model = out[0]
y_true = out[1]
y_pred = out[2]

acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

print(f"\nAccuracy: {acc}")
# print(f"F1-score: {f1}")

### Phase identification

In [ ]:
# Build pipeline with default preprocessing and feature extraction methods

from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.label_generator.label_generator import LabelGenerator
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Pipeline

from sklearn.svm import SVC


signal_types = [
    signals.Signals.ECG,
    # signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

label_gen = "phase"
signal_acq = SignalAcquisition(source_folder=SOURCE_FOLDER, signal_types=signal_types)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=100)
feature_extractor = FeatureExtractor()
label_generator = LabelGenerator(label_generation_method=label_gen)

model = SVC()
feature_selector = FeatureSelector(model, feature_names, num_features=3)
estimator_train = Estimator(0, model, name="SVC training")
estimator_test = Estimator(1, model, name="SVC testing")
estimator_train_val_test = Estimator(2, model, name="SVC train-val-test")

pipeline = Pipeline()


pipeline.generate_nodes_from_layers(
    # [signal_acq, signal_preprocessor, feature_extractor]
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train, estimator_test]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train_val_test]
)

out = pipeline.run()

# Results
# fitted_model = out[0]
y_true = out[1]
y_pred = out[2]

acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

print(f"\nAccuracy: {acc}")
# print(f"F1-score: {f1}")